读取 LogRecord

[read-LogRecord](./docs/images/read-logrecord.png)

上图信息是存储在数据文件中的一条 LogRecord 的结构，我们可以将其分成两部分：
- 一是头部信息，存储了一些元数据，例如 crc 校验值、Type 类型、Key 的大小、Value 的大小
- 二是包含用户实际的 key、value 部分

Header 部分

crc 占 4 个字节
Type 占 1 个字节
Key size 变长
Value size 变长

> key size 和 value size 变长的原因是为了节省空间，如果 key size 是 u32 类型的话，如果不使用变长，将固定占据4个字节，但是有的时候我们 key 的长度很小，例如长度为 5，那么只需要一个字节就够了

读取的时候，需要判断读取的偏移 + LogRecord 的最大头部字节数，是不是超过了文件大小，针对这个 case 需要特殊处理。

todo
# 新增测试用例
Put
    1. 正常put 一条数据
    2. 重复 Put key 相同的数据
    3. key 为空
    4. value 为空
    5. 写到数据文件进行了转换 工
    6. 重启后再 Put 数据
Get
     1. 正常读取一条数据
     2. 读取一个不存在的 key
     3. 值被重复 Put 后在读取
     4. 值被删除后再 Get
     5. 转换为了1旧日的数据文件，从日的数据文件上获取 value
     6. 重启后，前面写入的数据都能拿到
Delete
    1. 正常州除一个存在的 key
    2. 删除一个不存在的 key
    3. 删除一个空的 key值被删除之后重新 Put
    5. 重启之后，再进行校验

文件锁可互斥

Merge
    1. 没有任何数据的情况下进行 Merge
    2. 全部都是有效数据的情况下 Merge
    3. 有失效的数据，和被重复 Put 的数据
    4. 数据库中全部都是失效的数据
    5. Merge 的过程中有新的写入和删除
bug 修复：
    flock 文件不应该拷贝过去


# 索引尝试使用 B+ 树作为索引 （[boltdb](https://github.com/etcd-io/bbolt)）
- 问题，最新事务序列号的获取
  方案：
    - 可以从大到小倒序加载，获取到事务号（最坏情况会全量遍历文件）
    - 禁用 WriteBatch
    - Close 时，将最新的事务号写入到文件中(未调用 Close 时，会产生异常)

# 事务支持 MVCC 模式

# 索引粒度优化

当前内存中只有一个索引结构，所有的读取和写入都会竞争索引数据结构的锁

思路：把锁的粒度减小，使用多个索引结构，将 key 通过 hash 取模的方式，分配到不同的索引结构中

对迭代器的影响
- 每个索引结构内部是有序的，归并排序，建立最小堆，堆顶元素就是最小的元素，每次从堆顶取出一个元素，然后将该元素所在的索引结构的下一个元素放入堆中，然后重新调整堆，这样就能保证每次从堆顶取出的元素都是最小的元素，这样就能保证整体的有序性



# Write Batch 原子写的实现

实现事务（ACID）

- 原子性（Atomicity）：事务是一个原子操作单元，其对数据的修改，要么全都执行，要么全都不执行。
- 一致性（Consistency）：在事务开始之前和事务结束以后，数据库的完整性没有被破坏。
- 隔离性（Isolation）：数据库允许多个并发事务同时对其数据进行读和修改，隔离状态可以防止事务间的相互影响。
- 持久性（Durability）：事务完成之后，它对于数据的修改是永久性的，即使出现系统故障也能够保持。

## 如何实现原子性

原子性一般通过预写日志（WAL，Write Ahead Log）实现，例如 MySQL，使用到了 undo log 来保存事务回滚的信息，当事务提交失败后，可以通过 undo log 进行回滚，保证了事务的原子性。

## 如何实现隔离性

隔离性定义的标准几个隔离级别如下：
- 读未提交（Read Uncommitted）：允许脏读，也就是可能读取到其他会话中未提交事务修改的数据
- 读提交（Read Committed）：只能读取到已经提交的数据。 (不可重复读)
- 可重复读（Repeatable Read）：在开始读取数据（事务开启）时，不再允许修改操作。 (幻读)
- 串行化（Serializable）：完全串行化的读，每次读都需要获得表级共享锁，读写相互都会阻塞。

### 并发控制的实现

#### 两阶段锁（Two-Phase Locking）

在事务执行过程中，对需要操作的对象加锁，这样如果其它事务也需要操作同一个对象时，将会被阻塞（或者放弃），直到该事务释放锁，其它事务才能继续执行。

在类 LSM 的存储引擎中，一般不会采用两阶段锁来实现事务，因为 LSM 存储模型中，数据本身具有多版本的特征，因此更倾向于使用基于多版本的并发控制（MVCC）来实现事务。

#### 多版本并发控制（MVCC）

MVCC 是一种并发控制的方法，它为每个读写操作赋予一个唯一递增的时间戳，基于时间戳来判断数据的可见性，从而实现事务的隔离性。

即在修改数据时，不修改原数据，而是新增一条数据，并标识其版本


主流的数据库，例如 MySQL、PostgreSQL、Oracle 等都采用了 MVCC 来实现事务的隔离性。

基于MVCC 实现的事务隔离方式，一般叫做快照隔离（Snapshot Isolation，简称 SI），SI 并不是标准定义中事务的四种隔离级别，它的基本思路是每个事务都持有一个自己的快照，事务读取时会基于事务开始时的快照，其它事务的修改不会对读取有影响，当事务提交后，它的修改才会被其它事务看到。

SI 基本上解决了脏读、不可重复读、幻读的问题，但是它也引入了新的问题，即写偏向（偏斜）（Write Skew）问题。

[write-skew](./docs/images/write-skew.png)

基于此，后面有人提出了串行化快照隔离（Serializable Snapshot Isolation，简称 SSI），SI 只会检测到事务读取的数据是否被修改，而 SSI 会检测到事务读取的数据是否被修改，以及读取的数据是否满足某些约束条件，如果不满足，事务将会失败。

SSI 的大致实现思路：
写数据
- 将数据写入到内存中，并记录每个 key 到一个集合中，这个集合称为写集合（Write Set），便于冲突检测
- 提交事务
    - 加锁保证线程安全
    - 检测冲突，当前事务都去过的 key 是否被其它事务修改过，如果是，说明有冲突，事务放弃提交，回滚
    - 获取当前最新的事务序列号（序列号一般是全局递增的，每个事务都分配了一个序列号）
    - 将所有要写入的数据，key 进行编码，加上事务序列号
    - 将数据批量写到存储引擎中，保证原子性和持久性
    - 写完后更新内存索引
读数据
- 从当前事务的数据集合中获取，如果获取到返回
- 未获取到，则 key+当前事务的序列号，从存储引擎中查找
- 将读过的数据记录下来，便于在提交事务时进行冲突检测

## WriteBatch

暂时先使用全局锁，保证串行化

MVCC 参考资料
- https://zhuanlan.zhihu.com/p/395229054
- https://github.com/rfyiamcool/notes/blob/main/go_badger_transaction.md
- https://www.infoq.cn/article/teJA7X43BO2alp6rLCWk
- https://www.infoq.cn/article/KyZjpzySYHUYDJa2e1fS
- https://www.infoq.cn/article/gaOh3me9PmJBiQFD2j15
- https://catkang.github.io/2018/09/19/concurrency-control.html
- https://dgraph.io/blog/post/badger-txn/
- https://tech.ipalfish.com/blog/2020/03/26/isolation/



文件锁 flock
保证进程之间的互斥


持久化策略优化

当前默认是提供 sync，控制是否每一条的持久化到磁盘，当前默认是 false, 也就是把持久化这个行为交给了操作系统进行调度。

另外提供一个选项，用户可以设置积累到多少字节之后，再进行持久化，这样可以减少磁盘 IO 次数，提高性能。



启动速度优化
使用内存文件映射 MMap IO 来进行优化
mmap 是指将磁盘文件映射到内存地址空间，操作系统在读文件时，会触发缺页中断，然后将磁盘文件的数据读取到内存中，这样就可以避免用户态和内核态的切换，提高性能。


TODO： 数据文件布局优化
参考 LevelDB 或者 RocksDB 的 WAL 文件格式，将数据文件组织形式改为 block (32 KB) 的方式，加速数据的读取效率。

https://leveldb-handbook.readthedocs.io/zh/latest/journal.html
https://github.com/facebook/rocksdb/wiki/Write-Ahead-Log-%28WAL%29